In [46]:
import sys
import random
import pprint
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import typing
from typing import Tuple, List
import nltk
from nltk.corpus import reuters
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

In [47]:


# Our corpus uses the following start and end tokens.
START_TOKEN = '<START>'
END_TOKEN = '<END>'

np.random.seed(1337)
random.seed(1337)

Word Vectors are often used as a fundamental component for downstream NLP tasks, e.g. question answering, text generation, translation, etc. It is therefore important to build some intuition as to their strengths and weaknesses. Here, we will explore two types of word vectors: those derived from co-occurrence matrices, and those derived via GloVe.

The terms "word vectors" and "word embeddings" are often used interchangeably. The term "embedding" refers to the fact that we are encoding aspects of a word's meaning in a lower dimensional space. As Wikipedia states, "conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with a much lower dimension".

##### Part 1: Count-Based Word Vectors
Most word vector models start from the following idea:

*You shall know a word by the company it keeps (Firth, J. R. 1957:11)*

Many word vector implementations are driven by the idea that similar words, i.e., (near) synonyms, will be used in similar contexts. As a result, similar words will often be spoken or written along with a shared subset of words, i.e., contexts. By examining these contexts, we can try to develop embeddings for our words. With this intuition in mind, many "old school" approaches to constructing word vectors relied on word counts. Here we elaborate upon one of those strategies, co-occurrence matrices.

##### Co-Occurrence
A co-occurrence matrix counts how often things co-occur in some environment. Given some word 𝑤𝑖 occurring in the document, we consider the context window surrounding 𝑤𝑖. Supposing our fixed window size is 𝑛, then this is the 𝑛 preceding and 𝑛 subsequent words in that document, i.e. words 𝑤𝑖−𝑛…𝑤𝑖−1 and 𝑤𝑖+1…𝑤𝑖+𝑛. We build a co-occurrence matrix 𝑀, which is a symmetric word-by-word matrix in which 𝑀𝑖𝑗 is the number of times 𝑤𝑗 appears inside 𝑤𝑖's window among all documents.

The rows (or columns) of a co-occurrence matrix provide one type of word vectors (those based on word-word co-occurrence), but the vectors will be large in general (linear in the number of distinct words in a corpus). Thus, our next step is to run *dimensionality reduction*. In particular, we will run *SVD (Singular Value Decomposition)*, which is a kind of generalized *PCA (Principal Components Analysis)* to select the top 𝑘 *principal components*. Here's a visualization of dimensionality reduction with SVD. In this picture our co-occurrence matrix is 𝐴 with 𝑛 rows corresponding to 𝑛 words. We obtain a full matrix decomposition, with the singular values ordered in the diagonal 𝑆 matrix, and our new, shorter length-𝑘 word vectors in 𝑈𝑘.

![Picture of an SVD](./imgs/svd.png "SVD")

This reduced-dimensionality co-occurrence representation preserves semantic relationships between words, e.g. *doctor* and *hospital* will be closer than *doctor* and *dog*. 

In [63]:
def read_corpus(category: str) -> List[List[str]]:
    """ Read files from the specified Reuter's category.
        Params:
            category (string): category name
        Return:
            list of lists, with words from each of the processed files
    """
    files = reuters.fileids(category)
    return [[START_TOKEN] + [w.lower() for w in list(reuters.words(f))] + [END_TOKEN] for f in files]


def distinct_words(corpus: List[List[str]]) -> Tuple[List[str], int]:
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            corpus_words (list of strings): list of distinct words across the corpus, sorted
            num_corpus_words (integer): number of distinct words across the corpus
    """
    # Flatten list of lists.
    flattened_list = [word for document in corpus for word in document]
    
    # Remove duplicates.
    corpus_words = set(flattened_list)
    
    num_corpus_words = len(corpus_words)

    return sorted(list(corpus_words)), num_corpus_words

In [49]:
reuters_corpus = read_corpus(category="crude")
# pprint.pprint(reuters_corpus[:3], compact=True, width=100)

corpus_words, num_corpus_words = distinct_words(reuters_corpus)

In [50]:
# ---------------------
# Sanity check to ensure the correctness of distinct_words()
# ---------------------

# Define toy corpus
test_corpus = ["{} All that glitters isn't gold {}".format(START_TOKEN, END_TOKEN).split(" "), "{} All's well that ends well {}".format(START_TOKEN, END_TOKEN).split(" ")]
test_corpus_words, num_corpus_words = distinct_words(test_corpus)

# Correct answers
ans_test_corpus_words = sorted([START_TOKEN, "All", "ends", "that", "gold", "All's", "glitters", "isn't", "well", END_TOKEN])
ans_num_corpus_words = len(ans_test_corpus_words)

# Test correct number of words
assert(num_corpus_words == ans_num_corpus_words), "Incorrect number of distinct words. Correct: {}. Yours: {}".format(ans_num_corpus_words, num_corpus_words)

# Test correct words
assert (test_corpus_words == ans_test_corpus_words), "Incorrect corpus_words.\nCorrect: {}\nYours:   {}".format(str(ans_test_corpus_words), str(test_corpus_words))

# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

--------------------------------------------------------------------------------
Passed All Tests!
--------------------------------------------------------------------------------


In [134]:
def compute_co_occurrence_matrix(corpus: List[List[str]], window_size: int = 4) -> np.matrix:
    """ Compute co-occurrence matrix for the given corpus and window_size.    
        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (a symmetric numpy matrix of shape (number of unique words i n the corpus , number of unique words in the corpus)): 
                Co-occurence matrix of word counts. 
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, num_words = distinct_words(corpus)

    word2Ind = dict()    
    for idx, word in enumerate(words):
        word2Ind[word] = idx

    M = np.zeros((num_words, num_words))
    for document in corpus:
        for word_idx_in_doc, word in enumerate(document):
            indices_in_doc = list(range(word_idx_in_doc - window_size, word_idx_in_doc + window_size + 1))
            indices_in_doc = [x for x in indices_in_doc if x >= 0 and x <= (len(document)-1) and x != word_idx_in_doc]
            for i in indices_in_doc:
                matrix_idx_this_word = word2Ind[word]
                matrix_idx_neighbor = word2Ind[document[i]]
                M[(matrix_idx_this_word, matrix_idx_neighbor)] += 1

    return M, word2Ind

In [135]:
# ---------------------
# Sanity check to ensure the correctness of compute_co_occurrence_matrix()
# ---------------------

# Define toy corpus and get student's co-occurrence matrix
test_corpus = ["{} All that glitters isn't gold {}".format(START_TOKEN, END_TOKEN).split(" "), "{} All's well that ends well {}".format(START_TOKEN, END_TOKEN).split(" ")]
M_test, word2Ind_test = compute_co_occurrence_matrix(test_corpus, window_size=1)

# Correct M and word2Ind
M_test_ans = np.array( 
    [[0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,],
     [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,],
     [0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,],
     [0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,],
     [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,],
     [0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,],
     [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,],
     [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,],
     [0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,],
     [1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,]]
)
ans_test_corpus_words = sorted([START_TOKEN, "All", "ends", "that", "gold", "All's", "glitters", "isn't", "well", END_TOKEN])
word2Ind_ans = dict(zip(ans_test_corpus_words, range(len(ans_test_corpus_words))))

# Test correct word2Ind
assert (word2Ind_ans == word2Ind_test), "Your word2Ind is incorrect:\nCorrect: {}\nYours: {}".format(word2Ind_ans, word2Ind_test)

# Test correct M shape
assert (M_test.shape == M_test_ans.shape), "M matrix has incorrect shape.\nCorrect: {}\nYours: {}".format(M_test.shape, M_test_ans.shape)

# Test correct M values
for w1 in word2Ind_ans.keys():
    idx1 = word2Ind_ans[w1]
    for w2 in word2Ind_ans.keys():
        idx2 = word2Ind_ans[w2]
        student = M_test[idx1, idx2]
        correct = M_test_ans[idx1, idx2]
        if student != correct:
            print("Correct M:")
            print(M_test_ans)
            print("Your M: ")
            print(M_test)
            raise AssertionError("Incorrect count at index ({}, {})=({}, {}) in matrix M. Yours has {} but should have {}.".format(idx1, idx2, w1, w2, student, correct))

# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

<END> 0
<START> 1
All 2
All's 3
ends 4
glitters 5
gold 6
isn't 7
that 8
well 9
--------------------------------------------------------------------------------
Passed All Tests!
--------------------------------------------------------------------------------
